### Bike Sharing Assignment

In [ ]:
# Import required libraries

import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Suppress warnings

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Setting display options

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

#### - Reading and understanding the data

In [ ]:
# Importing the 'day.csv' file and saving it in the variable ''

bike_sharing = pd.read_csv('day.csv')

In [ ]:
# Viewing the data

bike_sharing.head()

In [ ]:
# Renaming the columns for easier understanding

bike_sharing.rename(columns = {'yr':'year','mnth':'month','temp':'temperature','hum':'humidity','cnt':'count'}, inplace = True)

bike_sharing.head()

In [ ]:
# Checking the shape of the dataframe

bike_sharing.shape

In [ ]:
# Summary of numeric columns

bike_sharing.describe()

In [ ]:
# Checking the datatypes of each column

bike_sharing.info()

In [ ]:
# Doing a check for null values
# As seen below, there aren't any null values present

bike_sharing.isnull().sum()

In [ ]:
# Mapping categorical columns with their categories as given in the data dictionary

bike_sharing['season'] = bike_sharing['season'].map({1: 'Spring', 2: 'Summer',3:'Fall', 4:'Winter' })

bike_sharing['month'] = bike_sharing['month'].map({1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'November',12:'December'})

bike_sharing['weathersit'] = bike_sharing['weathersit'].map({1: 'Clear',2:'Mist + Cloudy',3:'Light Snow',4:'Snow + Fog'})

bike_sharing['weekday'] = bike_sharing['weekday'].map({0:'Sunday',1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday',6:'Saturday'})

bike_sharing.head()

#### - Visualising the data

In [ ]:
# Identifying the categorical and continuous columns

bike_sharing.nunique().sort_values()

##### - As per my understanding based on the learning from the modules,
##### - Columns with values less than or equal to 40 are considered categorical
##### - Columns with values greater than 40 are considered continuous
##### - Sorting values in ascending order for easier readability

In [ ]:
# Now we can begin visualising the data for better understanding
# Visualising numerical columns using a pairplot

sns.pairplot(bike_sharing, vars = ['temperature','humidity','casual','windspeed','registered','atemp','count','instant'])
plt.show()

##### - By analysing the above pairplot we can see that some variables have a positive correlation to the count variable

In [ ]:
# Visualing the 7 categorical variables, we'll be using boxplots for this

plt.figure(figsize = (30, 15))

plt.subplot(2,4,1)
sns.boxplot(x = 'year', y = 'count', data = bike_sharing)

plt.subplot(2,4,2)
sns.boxplot(x = 'holiday', y = 'count', data = bike_sharing)

plt.subplot(2,4,3)
sns.boxplot(x = 'workingday', y = 'count', data = bike_sharing)

plt.subplot(2,4,4)
sns.boxplot(x = 'month', y = 'count', data = bike_sharing)
plt.xticks(rotation = 90)

plt.subplot(2,4,5)
sns.boxplot(x = 'season', y = 'count', data = bike_sharing)

plt.subplot(2,4,6)
sns.boxplot(x = 'weekday', y = 'count', data = bike_sharing)
plt.xticks(rotation = 45)

plt.subplot(2,4,7)
sns.boxplot(x = 'weathersit', y = 'count', data = bike_sharing)

plt.show()

##### - Bike rentals were considerably higher in 2019
##### - Bike rentals were higher during days which weren't holidays, indicating people use bikes for more than just leisure
##### - Bike rentals were much higher during the fall months
##### - Clear weather was the most preferred for bike rentals

#### - Taking a better look at the individual variables

In [ ]:
# Year
# 0:2018, 1:2019
# Rentals largest in 2019

sns.barplot('year', 'count', data = bike_sharing)
plt.show()

In [ ]:
# Month

plt.figure(figsize = (10,5))
sns.barplot('month', 'count',hue = 'year', data = bike_sharing)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
# Season
# Highest rentals in fall

sns.barplot('season', 'count', data = bike_sharing)
plt.show()

In [ ]:
# Weathersit
# Clear weather preferred

sns.barplot('weathersit', 'count', data = bike_sharing)
plt.show()

In [ ]:
# Humidity
# Higher the humidity, higher the number of bikes rented

sns.scatterplot(x = 'humidity', y = 'count', data = bike_sharing)
plt.show()

In [ ]:
# Temperature

sns.scatterplot(x = 'temperature', y = 'count', data = bike_sharing)
plt.show()

In [ ]:
# Checking correlation of variables using a heatmap

plt.figure(figsize = (20,10))
sns.heatmap(bike_sharing.corr(), cmap = 'Reds', annot = True)
plt.title('Correlation between individuals variables')
plt.show()

##### - instant, year, temperature, atemp, casual, registered are all highly correlated to count.
##### - A couple of other variables are also higly correlated

In [ ]:
# Dropping unnecessary columns
# instant dropped as its the unique value for each row
# dteday dropped as information already captured
# atemp dropped as highly correlated with temperature
# casual and registered dropped as they form the total, which is count

bike_sharing = bike_sharing.drop(['instant','dteday','atemp','casual','registered'], axis = 1)
bike_sharing.head()

#### - Data Preparation

In [ ]:
# Creating dummy variables for weathersit, month, weekday, season
# We'll also be dropping the first column

weather_sit = pd.get_dummies(bike_sharing['weathersit'], drop_first = True)
months = pd.get_dummies(bike_sharing['month'], drop_first = True)
weekdays = pd.get_dummies(bike_sharing['weekday'], drop_first = True)
seasons = pd.get_dummies(bike_sharing['season'], drop_first = True)

In [ ]:
# Adding the dummy variables to the original dataframe

bike_sharing = pd.concat([weather_sit,months,weekdays,seasons, bike_sharing], axis = 1)

In [ ]:
# Dropping weather_sit,months,weekdays,seasons as the dummy variables for these have been created

bike_sharing.drop(['weathersit','month','weekday','season'], axis = 1, inplace = True)

In [ ]:
# Checking the shape

bike_sharing.shape

In [ ]:
# Checking the correlation of the updated dataframe

plt.figure(figsize = (25,20))
sns.heatmap(bike_sharing.corr(), cmap = 'Reds', annot = True)
plt.show()

##### - From the above heatmap we can see that the months ranging from may to october, the summer season and temperature have a strong correlation with the target variable. 

#### - Dividing the data into train and test sets

In [ ]:
# Splitting into train and test sets

bike_sharing_train, bike_sharing_test = train_test_split(bike_sharing, train_size = 0.7, random_state = 100)

In [ ]:
# Checking shape after split

print(bike_sharing_train.shape)
print(bike_sharing_test.shape)

In [ ]:
# Scaling
# Methods to be used, standardization or normalisation via Min-Max scaling
# This is done so that the obtained coefficients are all on the same scale

scaler = MinMaxScaler()

# Creating a list of numeric variables

num_vars = ['temperature','humidity','windspeed','count']

# Fitting on data

bike_sharing_train[num_vars] = scaler.fit_transform(bike_sharing_train[num_vars])
bike_sharing_train.head()

In [ ]:
# Checking the variables after scaling
# All numeric variables are now between 0 and 1

bike_sharing_train.describe()

In [ ]:
# Checking correlation on the train

plt.figure(figsize = (25,20))
sns.heatmap(bike_sharing_train.corr(), cmap = 'Reds', annot = True)
plt.show()

##### - As seen above there is very little to no collinearity between the variables
##### - The highest correlation is between the count variable, year and temperature

In [ ]:
# Dividing the data into X and y sets
# Removing the target variable from y_train

y_train = bike_sharing_train.pop('count')
X_train = bike_sharing_train

#### - Bulding the linear model

In [ ]:
# Begin by using RFE or Recursive Feature Elimination
# Setting the output number to 15 variables as per what was indicated during the session

lm = LinearRegression()
lm.fit(X_train, y_train)

# Running RFE 

rfe = RFE(lm, 15)
rfe = rfe.fit(X_train,y_train)

In [ ]:
# List of variables selected

list(zip(X_train.columns, rfe.support_, rfe.ranking_))

In [ ]:
# RFE true columns

col = X_train.columns[rfe.support_]
col

In [ ]:
# RFE false columns

X_train.columns[~rfe.support_]

#### - Using statsmodel, for getting the summary stats

In [ ]:
# Creating X_test dataframe with the selected variables

X_train_rfe = X_train[col]

In [ ]:
# Creating the constant variable

X_train_rfe = sm.add_constant(X_train_rfe)

In [ ]:
# Running the model

lm = sm.OLS(y_train, X_train_rfe).fit()

print(lm.summary())

In [ ]:
# Dropping the constant

X_train_rfe = X_train_rfe.drop(['const'], axis = 1)

In [ ]:
# Calculating the VIFs

vif = pd.DataFrame()
X = X_train_rfe
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

##### - January can be dropped as it has a high p-value and a low VIF

In [ ]:
# Dropping January

X_train_new = X_train_rfe.drop(['January'], axis = 1)

In [ ]:
# Building the model without January

X_train_lm1 = sm.add_constant(X_train_new)
lm1 = sm.OLS(y_train, X_train_lm1).fit()
print(lm1.summary())

In [ ]:
# Let's drop the constant as well

X_train_lm1 = X_train_lm1.drop(['const'], axis = 1)

In [ ]:
# Calcualting new VIFs

vif = pd.DataFrame()
X = X_train_new
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

##### - Dropping humidity due to high VIF and low p-value

In [ ]:
# Dropping humidity

X_train_new2 = X_train_lm1.drop(['humidity'], axis = 1)

In [ ]:
# Building the model without humidity

X_train_lm2 = sm.add_constant(X_train_new2)
lm2 = sm.OLS(y_train, X_train_lm2).fit()
print(lm2.summary())

In [ ]:
# Dropping the constant

X_train_lm2 = X_train_lm2.drop(['const'], axis = 1)

In [ ]:
# Calcualting new VIFs

vif = pd.DataFrame()
X = X_train_new2
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

##### - Dropping holiday due to low p-value and high VIF

In [ ]:
# Dropping holiday

X_train_new3 = X_train_lm2.drop(['holiday'], axis = 1)

In [ ]:
# Building the model without holiday

X_train_lm3 = sm.add_constant(X_train_new3)
lm3 = sm.OLS(y_train, X_train_lm3).fit()
print(lm3.summary())

In [ ]:
# Dropping the constant

X_train_lm3 = X_train_lm3.drop(['const'], axis = 1)

In [ ]:
# Calcualting new VIFs

vif = pd.DataFrame()
X = X_train_new3
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

##### - Windspeed can be dropped due to high VIF and inverse correlation to the target variable.

In [ ]:
# Dropping windspeed

X_train_new4 = X_train_lm3.drop(['windspeed'], axis = 1)

In [ ]:
# Building the model without windspeed

X_train_lm4 = sm.add_constant(X_train_new4)
lm4 = sm.OLS(y_train, X_train_lm4).fit()
print(lm4.summary())

In [ ]:
# Dropping the constant

X_train_lm4 = X_train_lm4.drop(['const'], axis = 1)

In [ ]:
# Calcualting new VIFs

vif = pd.DataFrame()
X = X_train_new4
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

##### - Dropping July based on low p-value and low VIF

In [ ]:
# Dropping July

X_train_new5 = X_train_lm4.drop(['July'], axis = 1)

In [ ]:
# Building the model without windspeed

X_train_lm5 = sm.add_constant(X_train_new5)
lm5 = sm.OLS(y_train, X_train_lm5).fit()
print(lm5.summary())

In [ ]:
# Dropping the constant

X_train_lm5 = X_train_lm5.drop(['const'], axis = 1)

In [ ]:
# Calcualting new VIFs

vif = pd.DataFrame()
X = X_train_new5
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

##### - As we can see that all VIFs and p-values are acceptable, this will now be the model.
##### - The final model, X_train_lm5 was obtained by dropping January, July, humidity, holiday and windspeed variables.

#### - Residual Analysis

In [ ]:
# We need to do this analysis to check for normal distribution of the error terms.

In [ ]:
# Checking our model

X_train_lm5 = sm.add_constant(X_train_lm5)
X_train_lm5.head()

In [ ]:
# y pred training set

y_train_pred = lm5.predict(X_train_lm5)

In [ ]:
# Checking normal distribution of error terms

sns.distplot((y_train - y_train_pred), bins = 20)
plt.title('Error terms distribution')
plt.xlabel('Error values')
plt.show()

##### - As seem above the error terms are normally distributed

#### - Predictions

In [ ]:
# Performing scaling for numeric variables

num_vars = ['temperature','humidity','windspeed','count']

# Fitting on data

bike_sharing_test[num_vars] = scaler.transform(bike_sharing_test[num_vars])
bike_sharing_test.head()

In [ ]:
# Dividing test set into X and y

y_test = bike_sharing_test.pop('count')
X_test = bike_sharing_test
X_test.describe()

In [ ]:
X_train_lm5 = X_train_lm5.drop(['const'], axis = 1)

In [ ]:
X_train_lm5.columns

In [ ]:
# Using the model to make predictions

X_test_new = X_test[X_train_lm5.columns]

# Adding a constant

X_test_new1 = sm.add_constant(X_test_new)
X_test_new1.head()

In [ ]:
# Using the model to make predictions

y_pred = lm5.predict(X_test_new1)

#### - Calculating the R^2 and the Adjusted R^2 for the test set

In [ ]:
# Find R^2 for test set

r2_score(y_test,y_pred)

In [ ]:
# Find Adjusted R^2 for test set
# Applying the formula below

adj_r2 = 1 - (1 - 0.81150)*(11-1)/(11-1-1)
print(adj_r2)

#### - Model Evaluation

In [ ]:
# Understanding the spread by plotting together y_test and y_pred

plt.figure(figsize = (15,10))
plt.scatter(y_test, y_pred, color = 'Red')
plt.xlabel('y_test', fontsize = 15)
plt.ylabel('y_pred', fontsize = 15)
plt.show()

In [ ]:
# Using a regression plot to understand the fit on the test set

plt.figure(figsize = (15,10))
sns.regplot(y_test, y_pred, line_kws = {'color': 'red'})
plt.title('y_test vs y_pred', fontsize = 15)
plt.xlabel('y_test', fontsize = 15)
plt.ylabel('y_pred', fontsize = 15)
plt.show()

##### - Best fit line equation, y = mx + c + e
##### - Equation for the best fit line is as follows,
#####   count = 0.49 x temperature + 0.09 x September + 0.06 x Saturday + 0.05 x Summer + 0.097 x Winter + 0.23 x year + 0.056 x workingday - 0.03 x lightsnow - 0.078 x mistcloudy - 0.065 x Spring


#### - Result

##### - Train set R^2: 0.826
##### - Train adj R^2: 0.822
##### - Test set R^2: 0.8115
##### - Test set adj R^2: 0.7905
##### - Train and test R^2 difference: 1.5%
##### - Train and test adj R^2 difference: 3.15%
##### - Overall confirmed that the optimum model was obtained

#### - Interpretation

##### - The variable temperature had the highest coefficient at 0.49, meaning that an increase of a single unit in temperature would lead to an increase in bike rentals by 0.49 units.
##### - There were also some negative coefficents such as spring, mist + cloudy and light snow present.